In [ ]:
# Install OpenNMT-py 3.x
!pip3 install OpenNMT-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Open the folder where you saved your prepapred datasets from the first exercise
# You might need to mount your Google Drive first
%cd /content/drive/MyDrive/NLP2/fine_tuned/it_to_en/nmt/
!ls

/content/drive/.shortcut-targets-by-id/15ST_GbtQNVxa6lxoGBZxd99QBIwzy53D/NLP2/fine_tuned/it_to_en/nmt
compute-bleu.py
compute-bleu.py.1
config_brnn.yaml
config_trans.yaml
config.yaml
Europarl.en-it.en
Europarl.en-it.en-filtered.en
Europarl.en-it.en-filtered.en.subword
Europarl.en-it.en-filtered.en.subword.dev
Europarl.en-it.en-filtered.en.subword.test
Europarl.en-it.en-filtered.en.subword.test.desubword
Europarl.en-it.en-filtered.en.subword.train
Europarl.en-it.it
Europarl.en-it.it-filtered.it
Europarl.en-it.it-filtered.it.subword
Europarl.en-it.it-filtered.it.subword.dev
Europarl.en-it.it-filtered.it.subword.test
Europarl.en-it.it-filtered.it.subword.train
Europarl.en-it.xml
Europarl.en.translated
Europarl.en.translated2
Europarl.en.translated2.desubword
Europarl.en.translated.desubword
LICENSE
models
models_1l
models_2l
models_lstm
MT-Preparation
README
run
source.model
source.vocab
target.model
target.vocab
train.log


In [ ]:
# TRANSFORMER

config_trans = '''# config_trans.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: Europarl.en-it.it-filtered.it.subword.train
        path_tgt: Europarl.en-it.en-filtered.en.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: Europarl.en-it.it-filtered.it.subword.dev
        path_tgt: Europarl.en-it.en-filtered.en.subword.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: source.model
tgt_subword_model: target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.fren

# Stop training if it does not imporve after n validations
early_stopping: 3

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 5000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 3435

# Default: 100000 - Train the model to max n steps
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 25000

# Default: 10000 - Run validation after n steps
valid_steps: 500

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 2000
report_every: 100

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 4096   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 3
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 2
dec_layers: 2
heads: 8
hidden_size: 1024
word_vec_size: 1024
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config_trans.yaml", "w+") as config_trans_yaml:
  config_trans_yaml.write(config_trans)

In [ ]:
# Find the number of CPUs/cores on the machine
!nproc --all

4


In [ ]:
# Build Vocabulary

# -config: path to your config.yaml file
# -n_sample: use -1 to build vocabulary on all the segment in the training dataset
# -num_threads: change it to match the number of CPUs to run it faster

!onmt_build_vocab -config config_trans.yaml -n_sample -1 -num_threads 4

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-06-25 12:45:41,348 INFO] Counter vocab from -1 samples.
[2023-06-25 12:45:41,348 INFO] n_sample=-1: Build vocab on full datasets.
[2023-06-25 12:46:32,007 INFO] * Transform statistics for corpus_1(25.00%):
			* FilterTooLongStats(filtered=253)

[2023-06-25 12:46:32,212 INFO] * Transform statistics for corpus_1(25.00%):
			* FilterTooLongStats(filtered=238)

[2023-06-25 12:46:32,339 INFO] * Transform statistics for corpus_1(25.00%):
			* FilterTooLongStats(filtered=247)

[2023-06-25 12:46:32,819 INFO] * Transform statistics for corpus_1(25.00%):
			* FilterTooLongStats(filtered=260)

[2023-06-25 12:46:33,009 INFO] Counters src: 49429
[2023-06-25 12:46:33,009 INFO] Counters tgt: 48187
Traceback (most recent call last):
  File "/usr/local/bin/onmt_build_vocab", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/onmt/bin/build_vocab.py", line 283, in main
    build_vocab_main(o

In [ ]:
# Check if the GPU is active
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b6337c55-9052-ea3d-c9f3-86bce32541d3)


In [ ]:
# Check if the GPU is visable to PyTorch

import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

gpu_memory = torch.cuda.mem_get_info(0)
print("Free GPU memory:", gpu_memory[0]/1024**2, "out of:", gpu_memory[1]/1024**2)

True
Tesla T4
Free GPU memory: 14998.8125 out of: 15101.8125


In [ ]:
# Train the NMT model
# train 2 layer 512 hidden size
!onmt_train -config config_trans.yaml

[2023-06-24 15:01:36,385 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-06-24 15:01:37,033 INFO] Parsed 2 corpora from -data.
[2023-06-24 15:01:37,034 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-06-24 15:01:38,044 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', ',', '▁di', '.', "'", '▁e', '▁che']
[2023-06-24 15:01:38,045 INFO] The decoder start token is: <s>
[2023-06-24 15:01:38,045 INFO] Building model...
[2023-06-24 15:01:39,363 INFO] Switching model to float32 for amp/apex_amp
[2023-06-24 15:01:39,363 INFO] Non quantized layer compute is fp16
[2023-06-24 15:01:39,559 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(49440, 512, padding_idx=1)
        )
        (pe): PositionalEncoding()
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): 

In [ ]:
# Train the NMT model
# train 3 layer 256 hidden size
!onmt_train -config config_trans.yaml

[2023-06-25 08:37:05,962 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-06-25 08:37:05,963 INFO] Parsed 2 corpora from -data.
[2023-06-25 08:37:05,963 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-06-25 08:37:06,276 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', ',', '▁di', '.', "'", '▁e', '▁che']
[2023-06-25 08:37:06,277 INFO] The decoder start token is: <s>
[2023-06-25 08:37:06,277 INFO] Building model...
[2023-06-25 08:37:06,911 INFO] Switching model to float32 for amp/apex_amp
[2023-06-25 08:37:06,912 INFO] Non quantized layer compute is fp16
[2023-06-25 08:37:07,147 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(49440, 256, padding_idx=1)
        )
        (pe): PositionalEncoding()
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): 

In [ ]:
# Train the NMT model
# train 3 layer 256 hidden size
# learning rate 3
# warmup steps: 4000
!onmt_train -config config_trans.yaml

[2023-06-25 09:45:21,087 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-06-25 09:45:21,088 INFO] Parsed 2 corpora from -data.
[2023-06-25 09:45:21,089 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-06-25 09:45:21,386 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', ',', '▁di', '.', "'", '▁e', '▁che']
[2023-06-25 09:45:21,386 INFO] The decoder start token is: <s>
[2023-06-25 09:45:21,386 INFO] Building model...
[2023-06-25 09:45:21,983 INFO] Switching model to float32 for amp/apex_amp
[2023-06-25 09:45:21,983 INFO] Non quantized layer compute is fp16
[2023-06-25 09:45:22,218 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(49440, 256, padding_idx=1)
        )
        (pe): PositionalEncoding()
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): 

In [ ]:
# Train the NMT model
# train 4 layer 256 hidden size
# learning rate 3
# warmup steps: 2000
!onmt_train -config config_trans.yaml

[2023-06-25 11:12:35,292 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-06-25 11:12:35,293 INFO] Parsed 2 corpora from -data.
[2023-06-25 11:12:35,293 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-06-25 11:12:35,584 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', ',', '▁di', '.', "'", '▁e', '▁che']
[2023-06-25 11:12:35,585 INFO] The decoder start token is: <s>
[2023-06-25 11:12:35,585 INFO] Building model...
[2023-06-25 11:12:36,256 INFO] Switching model to float32 for amp/apex_amp
[2023-06-25 11:12:36,256 INFO] Non quantized layer compute is fp16
[2023-06-25 11:12:36,495 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(49440, 256, padding_idx=1)
        )
        (pe): PositionalEncoding()
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): 

In [ ]:
# Train the NMT model
# train 2 layer 1024 hidden size
# learning rate 3
# warmup steps: 2000
!onmt_train -config config_trans.yaml

[2023-06-25 12:48:01,021 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-06-25 12:48:01,022 INFO] Parsed 2 corpora from -data.
[2023-06-25 12:48:01,022 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-06-25 12:48:01,315 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', ',', '▁di', '.', "'", '▁e', '▁che']
[2023-06-25 12:48:01,315 INFO] The decoder start token is: <s>
[2023-06-25 12:48:01,315 INFO] Building model...
[2023-06-25 12:48:03,644 INFO] Switching model to float32 for amp/apex_amp
[2023-06-25 12:48:03,644 INFO] Non quantized layer compute is fp16
[2023-06-25 12:48:04,024 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(49440, 1024, padding_idx=1)
        )
        (pe): PositionalEncoding()
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer):